In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image
import glob
import os
import math
import csv
import os


In [2]:
lat_lon_file_path='/Users/moctader/Thesis_code/lat_lon/results.csv'
feature_vector_path='/Users/moctader/Thesis_code/lat_lon/feature_vector.csv'
csv_file_path = '/Users/moctader/Thesis_code/lat_lon/results.csv'
tiles_template = "/Users/moctader/Thesis/{t}/{z}/{x}/{y}.png"
combined_csv_path = '/Users/moctader/Thesis_code/lat_lon/combined_results.csv'
points = gpd.read_file("/Users/moctader/Thesis_code/GTK_ASsoil_obs.csv")
t_values = glob.glob('/Users/moctader/Thesis/*')
base_directory = "/Users/moctader/Thesis_code/output20/"


In [3]:
points.POINT_X = points.POINT_X.astype("float")
points.POINT_Y = points.POINT_Y.astype("float")
zoom_level = 10



In [5]:
samples = gpd.GeoDataFrame(
    points.CLASS, crs="EPSG:3067", geometry=gpd.points_from_xy(points.POINT_X, points.POINT_Y)
).to_crs("WGS84")

tile_list = [(point.x, point.y) for point in samples['geometry']]


In [6]:
samples["i"] = samples.index
samples["filenames"] = samples.apply(lambda row: f"{row['CLASS']}/image_{row['i']}", axis=1)



In [21]:
def get_lat_from_row(p):
    lon, lat = p.geometry.x, p.geometry.y
    return lat

def get_lon_from_one_column(geometry):
    # geometry is a column
    lon, lat = geometry.x, geometry.y
    return lon

In [22]:
samples["lat"] = samples.apply(lambda row: get_lat_from_row(row), axis=1)

In [23]:
samples["lon"] = samples["geometry"].map(get_lon_from_one_column)


In [25]:
def load_data(filename, directory):
    path = directory + "/" + filename + ".png"
    return np.array(Image.open(path))[:, :, :3]  # return 3 channels
    
    

In [26]:
# Get a list of file_names (files and directories) in the base directory
file_names = os.listdir(base_directory)

# Filter out non-directory file_names
files = [file for file in file_names if os.path.isdir(os.path.join(base_directory, file))]

for single_file in files:
    samples[single_file] = samples["filenames"].map(
        lambda name, directory=os.path.join(base_directory, single_file): load_data(name, directory)
    )


In [30]:
import numpy as np
import cv2
import pandas as pd


# Initialize an empty list to store combined images for all rows
combined_images_list = []

# Iterate over rows of the DataFrame
for index, row in samples.iterrows():
    # Initialize an empty list to store individual layers
    layers_list = []

    # Iterate over the selected columns
    for single_file in files:
        layer = (row[single_file])
        layers_list.append(layer)

    # Concatenate the layers along the third dimension to create a multi-channel image
    combined_image = np.concatenate(layers_list, axis=-1)
    
    # Append the combined image to the list
    combined_images_list.append(combined_image)

# Add the combined images as a new column in the DataFrame

samples["combined_channels"] = combined_images_list

# Print the updated DataFrame
print(samples.head(2))


  CLASS                   geometry  i    filenames        lat        lon  \
0   ASS  POINT (25.76894 64.77799)  0  ASS/image_0  64.777988  25.768938   
1   ASS  POINT (25.77630 64.79350)  1  ASS/image_1  64.793496  25.776304   

                                  elev_10m_hillshade  \
0  [[[197, 197, 197], [196, 196, 196], [197, 197,...   
1  [[[196, 196, 196], [197, 197, 197], [197, 197,...   

                                             landsat  \
0  [[[234, 121, 245], [163, 30, 166], [72, 10, 10...   
1  [[[113, 180, 121], [123, 190, 167], [133, 209,...   

                                      elev_10m_slope  \
0  [[[255, 255, 255], [254, 254, 254], [254, 254,...   
1  [[[255, 255, 255], [255, 255, 255], [255, 255,...   

                                  aem_real_component  \
0  [[[231, 24, 25], [231, 26, 26], [231, 24, 25],...   
1  [[[230, 25, 25], [230, 23, 22], [230, 21, 20],...   

                                   corine_land_cover  \
0  [[[3, 3, 3], [3, 3, 3], [25, 25, 25]

In [29]:
# Check the images chanels

base_directory = "/Users/moctader/Thesis_code/output20/"

# Get a list of file_names in the base directory
file_names = os.listdir(base_directory)

# Filter out non-directory file_names
files = [file for file in file_names if os.path.isdir(os.path.join(base_directory, file))]

for single_file in files:
    #mage_file_path=f'/Users/moctader/Thesis_code/output20/{single_file}/non-ASS/image_5786.png'
    image_file_path=f'/Users/moctader/Thesis_code/output20/{single_file}/ASS/image_0.png'
    image = Image.open(image_file_path)

    # Get the bands (channels) of the image
    bands = image.getbands()
    
    # Determine the number of channels
    num_channels = len(bands)

    # Print the result
    print(f"The image at {image_file_path} has {num_channels} channel(s): {bands}")

The image at /Users/moctader/Thesis_code/output20/elev_10m_hillshade/ASS/image_0.png has 3 channel(s): ('R', 'G', 'B')
The image at /Users/moctader/Thesis_code/output20/landsat/ASS/image_0.png has 3 channel(s): ('R', 'G', 'B')
The image at /Users/moctader/Thesis_code/output20/elev_10m_slope/ASS/image_0.png has 3 channel(s): ('R', 'G', 'B')
The image at /Users/moctader/Thesis_code/output20/aem_real_component/ASS/image_0.png has 3 channel(s): ('R', 'G', 'B')
The image at /Users/moctader/Thesis_code/output20/corine_land_cover/ASS/image_0.png has 3 channel(s): ('R', 'G', 'B')
The image at /Users/moctader/Thesis_code/output20/elev_10m/ASS/image_0.png has 3 channel(s): ('R', 'G', 'B')
The image at /Users/moctader/Thesis_code/output20/aem_apparent_resistivity/ASS/image_0.png has 3 channel(s): ('R', 'G', 'B')
The image at /Users/moctader/Thesis_code/output20/aem_imaginary_component/ASS/image_0.png has 3 channel(s): ('R', 'G', 'B')
The image at /Users/moctader/Thesis_code/output20/TPI/ASS/image